In [ ]:
# notebook debug only
from pyspark.sql.functions import col,from_json, lpad, year, month, dayofmonth
from pyspark.sql import SparkSession,Row
# spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

#read json from text file
dfFromTxt=spark.read.format("json").option("inferschema",True).text("s3://bots-taijitu-intermediate-data-export/example.ddb.json.txt")

selectedColumns = ["id", "appId", "channel", "checkoutId", "creationTimestamp", "currency", "forwardedFor", "geolocation", "launchId", "launchSkuStatus", 
                   "launchSkuStatusValidation", "locale", "paymentStatus", "paymentToken", "postpayLink", "postpayRetry", "rank", "reentryPermitted", 
                   "reservationId", "reservationJobId", "reservationTimestamp", "retailPickupPerson", "selectedTimestamp", "skuId", "sort", "status", 
                   "upmId", "ttl", "traceContext", "trueClientIp", "userAgent", "userType", "waitingReason", "validationSummary_result", 
                   "shipping_recipient_firstName", "shipping_recipient_lasttName", "shipping_recipient_phoneNumber",
                   "shipping_address_address1", "shipping_address_city", "shipping_address_state", "shipping_address_postalCode", "shipping_address_country", "shipping_address_county",
                   "shipping_method",
                   "year", "month", "day",
                   "metadata_eventId", "metadata_eventName", "metadata_tableName", 
                   "db_item_creationDateTime", "db_item_key"
                  ]

dfJSON = dfFromTxt.withColumn("jsonData",from_json(col("value"),schema))\
    .withColumn("sh", from_json(col("jsonData.dynamodb.NewImage.shipping.S"), shippingSchema))\
    .withColumn("valSummary", from_json(col("jsonData.dynamodb.NewImage.validationSummary.S"), validationSummarySchema)) \
    .withColumn("metadata_eventId", col("jsonData.eventID"))\
    .withColumn("metadata_eventName", col("jsonData.eventName"))\
    .withColumn("metadata_tableName", col("jsonData.tableName"))\
    .withColumn("db_item_creationDateTime", col("jsonData.dynamodb.ApproximateCreationDateTime"))\
    .withColumn("db_item_key", col("jsonData.dynamodb.Keys.id.S"))\
    .withColumn("id", col("jsonData.dynamodb.NewImage.id.S"))\
    .withColumn("appId", col("jsonData.dynamodb.NewImage.appId.S"))\
    .withColumn("channel", col("jsonData.dynamodb.NewImage.channel.S"))\
    .withColumn("checkoutId", col("jsonData.dynamodb.NewImage.checkoutId.S"))\
    .withColumn("creationTimestamp", col("jsonData.dynamodb.NewImage.creationTimestamp.N"))\
    .withColumn("currency", col("jsonData.dynamodb.NewImage.currency.S"))\
    .withColumn("forwardedFor", col("jsonData.dynamodb.NewImage.forwardedFor.S"))\
    .withColumn("geolocation", col("jsonData.dynamodb.NewImage.geolocation.S"))\
    .withColumn("launchId", col("jsonData.dynamodb.NewImage.launchId.S"))\
    .withColumn("launchSkuStatus", col("jsonData.dynamodb.NewImage.launchSkuStatus.S"))\
    .withColumn("launchSkuStatusValidation", col("jsonData.dynamodb.NewImage.launchSkuStatusValidation.S"))\
    .withColumn("locale", col("jsonData.dynamodb.NewImage.locale.S"))\
    .withColumn("paymentStatus", col("jsonData.dynamodb.NewImage.paymentStatus.S"))\
    .withColumn("paymentToken", col("jsonData.dynamodb.NewImage.paymentToken.S"))\
    .withColumn("postpayLink", col("jsonData.dynamodb.NewImage.postpayLink.S"))\
    .withColumn("postpayRetry", col("jsonData.dynamodb.NewImage.postpayRetry.BOOL"))\
    .withColumn("rank", col("jsonData.dynamodb.NewImage.rank.N"))\
    .withColumn("reentryPermitted", col("jsonData.dynamodb.NewImage.reentryPermitted.BOOL"))\
    .withColumn("reservationId", col("jsonData.dynamodb.NewImage.reservationId.S"))\
    .withColumn("reservationJobId", col("jsonData.dynamodb.NewImage.reservationJobId.S"))\
    .withColumn("reservationTimestamp", col("jsonData.dynamodb.NewImage.reservationTimestamp.N"))\
    .withColumn("retailPickupPerson", col("jsonData.dynamodb.NewImage.retailPickupPerson.S"))\
    .withColumn("selectedTimestamp", col("jsonData.dynamodb.NewImage.selectedTimestamp.N"))\
    .withColumn("skuId", col("jsonData.dynamodb.NewImage.skuId.S"))\
    .withColumn("sort", col("jsonData.dynamodb.NewImage.sort.N"))\
    .withColumn("status", col("jsonData.dynamodb.NewImage.status.S"))\
    .withColumn("traceContext", col("jsonData.dynamodb.NewImage.traceContext.S"))\
    .withColumn("trueClientIp", col("jsonData.dynamodb.NewImage.trueClientIp.S"))\
    .withColumn("ttl", col("jsonData.dynamodb.NewImage.ttl.N"))\
    .withColumn("upmId", col("jsonData.dynamodb.NewImage.upmId.S"))\
    .withColumn("userAgent", col("jsonData.dynamodb.NewImage.userAgent.S"))\
    .withColumn("validationSummary_result", col("jsonData.dynamodb.NewImage.validationSummary.S"))\
    .withColumn("waitingReason", col("jsonData.dynamodb.NewImage.waitingReason.S"))\
    .withColumn("userType", col("jsonData.dynamodb.NewImage.userType.S"))\
    .withColumn("validationSummary_result", col("valSummary.result"))\
    .withColumn("shipping_recipient_firstName", col("sh.recipient.firstName"))\
    .withColumn("shipping_recipient_lasttName", col("sh.recipient.lastName"))\
    .withColumn("shipping_recipient_phoneNumber", col("sh.recipient.phoneNumber"))\
    .withColumn("shipping_address_address1", col("sh.address.address1"))\
    .withColumn("shipping_address_city", col("sh.address.city"))\
    .withColumn("shipping_address_state", col("sh.address.state"))\
    .withColumn("shipping_address_postalCode", col("sh.address.postalCode"))\
    .withColumn("shipping_address_country", col("sh.address.country"))\
    .withColumn("shipping_address_county", col("sh.address.county"))\
    .withColumn("shipping_method", col("sh.method"))\
    .withColumn("year", year((col("jsonData.dynamodb.NewImage.creationTimestamp.N")/ 1000).cast(TimestampType())))\
    .withColumn("month", lpad(month((col("jsonData.dynamodb.NewImage.creationTimestamp.N")/ 1000).cast(TimestampType())), 2, "0"))\
    .withColumn("day", lpad(dayofmonth((col("jsonData.dynamodb.NewImage.creationTimestamp.N")/ 1000).cast(TimestampType())), 2, "0"))\
    .select(selectedColumns)
# dfJSON.printSchema()
dfJSON.show(truncate=False)